In [ ]:
# Goal
# Start with dataset
  # Could generate this dataset or pre-existing
  # Could label this dataset manually or have it zero-shot labelled
# Build a custom text classifier on labelled data
  # Test text classifier on labelled data vs zero-shot model

What happens if I put a markdown cell here, is that easy to tell apart?

In [ ]:
try:
  import datasets, evaluate, accelerate
except:
  !pip install datasets
  !pip install accelerate -U
  !pip install evaluate
  import datasets, evaluate, accelerate

from datasets import Dataset

import random
import pandas as pd

import transformers
from google.colab import drive
drive.mount('/content/drive')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 14.7 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.4/302.4 kB 6.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none

Even a markdown cell here, how does this look?

In [ ]:
file_path = "/content/drive/MyDrive/nutrify/coyo700m/food_not_food_captions/food_not_food_captions_rows_1000000.csv"

df = pd.read_csv(file_path)
df.head()

,id,url,text,width,height,image_phash,text_length,word_count,num_tokens_bert,num_tokens_gpt,...,clip_similarity_vitb32,clip_similarity_vitl14,nsfw_score_opennsfw2,nsfw_score_gantman,watermark_score,aesthetic_score_laion_v2,text_food_not_food_labels,text_food_not_food_scores,text_food_not_food_label,text_food_not_food_score
0,68719569143,https://m.media-amazon.com/images/I/513i2gEKSK...,MUSCLETECH Amino NRG SX-7 Revolution Ultimate ...,353,500,af99d06cd0e2c31d,97,16,26,29,...,0.404541,0.375977,0.000806,0.014266,0.002690,4.147951,"['food', 'not_food']","[0.6513671875, 0.348388671875]",food,0.651367
1,5274220575161,https://therapydiaportland.com/wp-content/uplo...,9 yoga poses for runners therapydia portland,470,353,ac3c3b73664c6562,44,7,8,10,...,0.285645,0.264648,0.003700,0.078907,0.007236,4.882448,"['not_food', 'food']","[0.9609375, 0.039337158203125]",not_food,0.960938
2,7962869880803,https://cdn.fashiola.ph/L43220811/ted-baker-qi...,Ted Baker Qinala heeled ankle boot with T bran...,300,450,b93687dbc8cca4d0,60,11,13,14,...,0.318115,0.261230,0.006603,0.024137,0.032537,5.102950,"['not_food', 'food']","[0.99267578125, 0.00788116455078125]",not_food,0.992676
3,3427384496754,https://photos.classiccars.com/cc-temp/listing...,1947 Plymouth Street Rod (CC-1206989) for sale...,1280,960,8e44d53b6bc614ea,68,11,19,18,...,0.315674,0.323730,0.005131,0.039114,0.000400,4.575576,"['not_food', 'food']","[0.99365234375, 0.006145477294921875]",not_food,0.993652
4,2611340287260,https://cdn.shopify.com/s/files/1/0425/2244/39...,"Five Nights at Freddy's, Fazbear Frights #1, I...",300,300,a9a586dd8696985d,57,11,19,17,...,0.336914,0.264893,0.004673,0.006453,0.002217,3.851733,"['not_food', 'food']","[0.96337890625, 0.036834716796875]",not_food,0.963379


And another markdown cell here, can you tell code part from markdown? 

In [ ]:
# How many rows contain "menu"
def contains_menu(text):
  if "menu" in text:
    return True
  else:
    return False

df["menu_in_text"] = df["text"].apply(lambda x: contains_menu(x))
df["menu_in_text"].value_counts()

menu_in_text
False    999637
True        363
Name: count, dtype: int64

In [ ]:
df["text_food_not_food_label"].value_counts()

text_food_not_food_label
not_food    935125
food         64875
Name: count, dtype: int64

In [ ]:
# How many food samples have a score over X
scores = [0.65, 0.70, 0.75, 0.80, 0.85, 0.90, 0.95, 0.97, 0.98, 0.99]
for score in scores:
    print(f'Number of rows over {score}: {len(df[(df["text_food_not_food_label"] == "food") & (df["text_food_not_food_score"] > score)])}')

Number of rows over 0.65: 60006
Number of rows over 0.7: 58333
Number of rows over 0.75: 56501
Number of rows over 0.8: 54560
Number of rows over 0.85: 52322
Number of rows over 0.9: 49524
Number of rows over 0.95: 44985
Number of rows over 0.97: 41839
Number of rows over 0.98: 39135
Number of rows over 0.99: 34326


In [ ]:
# Create a dataset of food rows over 0.85 + 100k random "not_food" rows
df_food_rows_sample = df[(df["text_food_not_food_label"] == "food") & (df["text_food_not_food_score"] >= 0.85)]
len(df_food_rows_sample)

52322

In [ ]:
# Create a sample of 100k "not_food" rows
df_not_food_rows_sample = df[df["text_food_not_food_label"] == "not_food"].sample(100_000)
len(df_not_food_rows_sample)

100000

In [ ]:
df_subset = pd.concat([df_food_rows_sample, df_not_food_rows_sample]).reset_index(drop=True)
len(df_subset)

152322

In [ ]:
for row in df_subset.sample(5).iterrows():
  row_item = row[1]
  text = row_item["text"]
  food_label = row_item["text_food_not_food_label"]
  food_score = row_item["text_food_not_food_score"]
  print(f"Text: {text}")
  print(f"Food label: {food_label}")
  print(f"Food score: {food_score}")
  print(f"------")
  print()

Text: Deep Red - Audio commentary with Argento expert Thomas Rostock
Food label: not_food
Food score: 0.9609375
------

Text: Young shaggy scalycap mushrooms (<B>Pholiota squarrosa</B>, Russian name Cheshuychatka) in Lisiy Nos, west from Saint Petersburg. Russia, <A HREF="../date-en/2016-09-12.htm">September 12, 2016</A
Food label: food
Food score: 0.99609375
------

Text: Picture of Belgian Chocolate Unicorn Lolly
Food label: food
Food score: 0.99267578125
------

Text: Simply Shaker 7050 Baby Spinach
Food label: food
Food score: 0.998046875
------

Text: A basket of lilies with teddy bear
Food label: not_food
Food score: 0.99365234375
------



In [ ]:
# See here: https://huggingface.co/docs/datasets/en/loading#python-list-of-dictionaries
# Make a list of dicts for HF Dataset
data_dict = df_subset[["text", "text_food_not_food_label"]].to_dict(orient="records")
data_dict_list = []
for item in data_dict:
  item_text = item["text"]
  item_label = 0 if item["text_food_not_food_label"] == "not_food" else 1
  data_dict_list.append({item_label: item_text})

import random
random.sample(data_dict_list, k=5)

[{1: 'a bottle of Miracle Whip'},
 {1: 'Croquetas de Pollo Recipe (Chicken Croquettes): very popular as party food and street food, also common at Dominican Christmas dinner tables.'},
 {0: 'How to Build a Radiator Cover This easy DIY tutorial shows you how to make a radiator cover to cover those unsightly or unused radiators you might have in your home. Diy Radiator Cover, Radiator Screen, Radiator Ideas, Home Radiators, Baseboard Heater Covers, Diy Heater, Modern Birdhouses, Small Bedroom Storage, Small Bedrooms'},
 {0: 'Anthony and Olivia&apos;s wedding in Vancouver, British Columbia 7'},
 {0: 'Nissan Micra 2018 - Image #5'}]

In [ ]:
data_dict = {col_name: df_subset[col_name].tolist() for col_name in ["text", "text_food_not_food_label"]}

dataset = Dataset.from_dict(data_dict)
dataset

Dataset({
    features: ['text', 'text_food_not_food_label'],
    num_rows: 152322
})

In [ ]:
# Rename column, see; https://huggingface.co/docs/datasets/en/process#rename
dataset = dataset.rename_column("text_food_not_food_label", "label")
dataset

Dataset({
    features: ['text', 'label'],
    num_rows: 152322
})

In [ ]:
# Turn labels into 0 or 1 (e.g. 0 for "not_food", 1 for "food"), see: https://huggingface.co/docs/datasets/en/process#map
def map_labels_to_number(example):
  example["label"] = 0 if example["label"] == "not_food" else 1
  return example

dataset = dataset.map(map_labels_to_number)
dataset[:5]

Map:   0%|          | 0/152322 [00:00<?, ? examples/s]

{'text': ['The expression of copra farmers',
  'Super easy cream biscuits made with just TWO ingredients from TheSuburbanSoapbox.com.',
  'No Gluten Pizza - The Perfect Vegan Pie For All Occasions',
  ":::FROSTY'S HOT COCO BAR SIGN:::",
  'Zucchini Escabeche @ A Moveable Feast Catering'],
 'label': [1, 1, 1, 1, 1]}

In [ ]:
dataset.shuffle()[:5]

{'text': ["Beatles Sgt Pepper Apple Label Stereo Analog '69 3rd Press VG++ ULTRASONIC Clean",
  'GMC Savana Commercial Cutaway 3500 ** Cube 12 pieds Deck ** RAMPE ** COMME NEUF 2020',
  'Image no. 1 for Solute Carrier Family 25 (Mitochondrial Carrier, Adenine Nucleotide Translocator), Member 4 (SLC25A4) ELISA Kit (ABIN5524005',
  'BAPE x Coach SS20 Collection Revealed',
  'Composition XIV by Conrad Marca-Relli at'],
 'label': [0, 0, 0, 0, 0]}

In [ ]:
# Create train/test splits, see: https://huggingface.co/docs/datasets/en/process#split
dataset = dataset.train_test_split(test_size=0.2)

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 121857
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 30465
    })
})

In [ ]:
dataset["train"][0], dataset["test"][0]

({'text': 'Follow Your Dreams Floral LastingLite Sleeve', 'label': 0},
 {'text': 'Blue & Red Basketball TMAC Millennium Boost Tracy McGrady Sneakers',
  'label': 0})

## Preprocess

See: https://huggingface.co/docs/transformers/en/tasks/sequence_classification#preprocess

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")

def preprocess_function(examples):
  return tokenizer(examples["text"], truncation=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
tokenized_dataset = dataset.map(preprocess_function, batched=True)
tokenized_dataset

Map:   0%|          | 0/121857 [00:00<?, ? examples/s]

Map:   0%|          | 0/30465 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 121857
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 30465
    })
})

In [ ]:
tokenized_dataset["train"][0], tokenized_dataset["test"][0]

({'text': 'Follow Your Dreams Floral LastingLite Sleeve',
  'label': 0,
  'input_ids': [101, 3582, 2115, 5544, 18686, 9879, 22779, 10353, 102],
  'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]},
 {'text': 'Blue & Red Basketball TMAC Millennium Boost Tracy McGrady Sneakers',
  'label': 0,
  'input_ids': [101,
   2630,
   1004,
   2417,
   3455,
   1056,
   22911,
   10144,
   12992,
   10555,
   11338,
   16307,
   2100,
   28130,
   102],
  'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]})

In [ ]:
# Collate examples and pad them each batch
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
data_collator

DataCollatorWithPadding(tokenizer=DistilBertTokenizerFast(name_or_path='distilbert/distilbert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}, padding=True, max_length=None, pad_to_multiple_of=None, return_tensor

## Evaluation

See: https://huggingface.co/docs/transformers/en/tasks/sequence_classification#evaluate

In [ ]:
import evaluate
import numpy as np

accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
  predictions, labels = eval_pred
  predictions = np.argmax(predictions, axis=1)
  return accuracy.compute(predictions=predictions, references=labels)

## Train

See: https://huggingface.co/docs/transformers/en/tasks/sequence_classification#train

3 steps for training:

1. Define model
2. Define training arguments
3. Pass training arguments to Trainer
4. Call `train()`

In [ ]:
# Create mapping from id2label and label2id
id2label = {0: "not_food", 1: "food"}
label2id = {"not_food": 0, "food": 1}

In [ ]:
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments

model = AutoModelForSequenceClassification.from_pretrained(
    pretrained_model_name_or_path="distilbert/distilbert-base-uncased",
    num_labels=2,
    id2label=id2label,
    label2id=label2id
)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Create training arguments
# See: https://huggingface.co/docs/transformers/v4.40.2/en/main_classes/trainer#transformers.TrainingArguments
training_args = TrainingArguments(
    output_dir="food_not_food_text_model",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    # push_to_hub=True
)

In [ ]:
# Setup Trainer
# Note: Trainer applies dynamic padding by default when you pass `tokenizer` to it.
# In this case, you don't need to specify a data collator explicitly.
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    #data_collator=data_collator,
    compute_metrics=compute_metrics
)

In [ ]:
# Batch size 16
#  [ 391/15234 00:22 < 14:27, 17.12 it/s, Epoch 0.05/2]

# Batch size 32
# [ 724/7618 01:08 < 10:51, 10.58 it/s, Epoch 0.19/2]

# Batch size 64
#  [ 150/3810 00:31 < 12:52, 4.74 it/s, Epoch 0.08/2]

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.111100,0.095816,0.963466
2,0.053100,0.123158,0.964484


TrainOutput(global_step=7618, training_loss=0.08334020205012267, metrics={'train_runtime': 831.381, 'train_samples_per_second': 293.144, 'train_steps_per_second': 9.163, 'total_flos': 4317340976716644.0, 'train_loss': 0.08334020205012267, 'epoch': 2.0})

In [ ]:
# Optional: push the model to Hugging Face Hub for re-use later
# Note: Requires Hugging Face login
# trainer.push_to_hub()

In [ ]:
# Save model
# See: https://discuss.huggingface.co/t/how-to-save-my-model-to-use-it-later/20568/4
trainer.save_model("food_not_food_text_model")

## Inference

Making predictions on our own text options.

See: https://huggingface.co/docs/transformers/en/tasks/sequence_classification#inference

In [ ]:
sample_text = "A delicious photo of a plate of scrambled eggs, bacon and toast"

### Pipeline mode

In [ ]:
from transformers import pipeline

food_not_food_classifier = pipeline(task="text-classification", model="./food_not_food_text_model")
food_not_food_classifier(sample_text)

[{'label': 'food', 'score': 0.9997596144676208}]

In [ ]:
sample_text_not_food = "A yellow tractor driving over the hill"
food_not_food_classifier(sample_text_not_food)

[{'label': 'not_food', 'score': 0.999407172203064}]

In [ ]:
# Predicting works with lists
# Can find the examples with highest confidence and keep those
sentences = [
    "I whipped up a fresh batch of code, but it seems to have a syntax error.",
    "We need to marinate these ideas overnight before presenting them to the client.",
    "The new software is definitely a spicy upgrade, taking some time to get used to.",
    "Her social media post was the perfect recipe for a viral sensation.",
    "He served up a rebuttal full of facts, leaving his opponent speechless.",
    "The team needs to simmer down a bit before tackling the next challenge.",
    "Our budget is a bit thin, so we'll have to use budget-friendly materials for this project.",
    "The presentation was a delicious blend of humor and information, keeping the audience engaged.",
    "I'm feeling overwhelmed by this workload – it's a real information buffet.",
    "We're brainstorming new content ideas, hoping to cook up something innovative."
]

food_not_food_classifier(sentences)

[{'label': 'not_food', 'score': 0.9527214169502258},
 {'label': 'not_food', 'score': 0.952194094657898},
 {'label': 'not_food', 'score': 0.8402552008628845},
 {'label': 'food', 'score': 0.9794776439666748},
 {'label': 'not_food', 'score': 0.994716465473175},
 {'label': 'food', 'score': 0.6270619034767151},
 {'label': 'not_food', 'score': 0.9927234053611755},
 {'label': 'food', 'score': 0.779589831829071},
 {'label': 'not_food', 'score': 0.7383647561073303},
 {'label': 'food', 'score': 0.7064051628112793}]

### PyTorch mode

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("food_not_food_text_model")
inputs = tokenizer(sample_text, return_tensors="pt")

In [ ]:
import torch
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("food_not_food_text_model")
with torch.no_grad():
  logits = model(**inputs).logits

In [ ]:
# Get predicted class
predicted_class_id = logits.argmax().item()
print(f"Text: {sample_text}")
print(f"Predicted label: {model.config.id2label[predicted_class_id]}")

Text: A delicious photo of a plate of scrambled eggs, bacon and toast
Predicted label: food
